<img src="./images/DLI_Header.png" style="width: 400px;">

# 합성곱신경망 (Convolutional Neural Networks)

이전 섹션에서 우리는 ASL 이미지를 분류하는 간단한 모델을 설계하고 학습했습니다. 모델은 매우 높은 정확도로 학습(Training) 데이터를 올바르게 분류하였지만, 검증(Validation) 데이터셋에서는 잘 작동하지 않았습니다. 학습하지 않은 데이터를 잘 일반화하지 못하는 현상을 [과적합](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html)이라고 부르며, 이번 섹션에서 우리는 이미지를 읽고 분류하는데 자주 사용되는 [합성곱신경망(CNN)](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) 이라고 불리는 모델을 배울 예정입니다.

## 목표

* CNN 모델을 위한 데이터 준비
* 세련된 형태의 CNN 모델 생성, 다양한 layer들의 이해
* CNN 모델 학습 및 성능 관찰

## 데이터 준비 및 로드

아래 셀은 이전 lab에서 학습한 데이터 전처리 기술이 포함되어 있습니다. 다음 단계로 가기 이전 실행하신 후 검토하십시오.

In [1]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## CNN을 위한 이미지 변환(Reshaping)

마지막 연습문제에서, 데이터셋의 개별 형식은 일렬로 된 784픽셀 형식이었습니다.

In [2]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

이러한 형식은 서로 가까운 픽셀에 대한 모든 정보를 가지고 있지 않습니다. 그렇기 때문에, feature를 탐지하는 컨볼루션(convolution)을 적용할 수 없습니다. 데이터 셋을 28x28픽셀 형식으로 변경해 보겠습니다. 이를 통해 컨볼루션에서 인접한 픽셀끼리 그룹을 짓고 중요한 feature들을 탐지할 수 있습니다.

모델의 첫 번째 컨볼루션 레이어의 경우 이미지의 높이와 너비뿐만 아니라 [채널](https://www.photoshopessentials.com/essentials/rgb/)의 숫자도 요구된다는 점에 유의해야합니다. 우리의 이미지는 Grayscale이기 때문에 채널은 1의 값을 가져야합니다.

즉, 현재의 shape인 (27455, 784)을 (27455, 28, 28, 1)로 변환할 필요가 있습니다. 편의상, 우리는 동일하게 유지하기를 원하는 모든 차원에 대해 [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape)을 진행하는 경우 '-1'을 통해 표현할 수 있으므로 다음과 같이 구성할 수 있습니다.

In [3]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [4]:
x_train.shape

(27455, 28, 28, 1)

In [5]:
x_valid.shape

(7172, 28, 28, 1)

In [6]:
x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

## 합성곱 모델 생성

오늘날, 많은 데이터 과학자는 유사한 프로젝트를 기반으로, 자신만의 프로젝트를 시작합니다. 문제가 완전히 새로운 형태가 아닌 경우, 사람들은 [TensorFlow Hub](https://www.tensorflow.org/hub)나 [NGC Catalog](https://ngc.nvidia.com/catalog/models) 와 같은 온라인 저장소에 게시된 모델과 유사한 모델을 활용할 가능성이 높습니다. 오늘, 우리는 이 문제에 다음과 같은 형태의 모델을 활용할 것입니다.

<img src="images/cnn.png" width=180 />

우리는 강의에서 다양한 종류의 레이어들을 이미 공부했고, 여기서도 공식문서를 나타내는 링크와 함께 여기 있는 모든 레이어들을 학습할 것입니다. 의문 사항이 생길 경우 공식 문서를 읽거나 [stack overflow](https://stackoverflow.com/)에 문의하십시오.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<img src="images/conv2d.png" width=300 />

2D 컨볼루션 레이어입니다. 작은 커널은 입력 이미지들을 훑으며, 분류에 중요한 특징들을 파악하게 됩니다. 모델의 초기 컨볼루션은 선과 같은 간단한 특징을 탐지하게 됩니다. 그 이후 컨볼루션은 점점 더 복잡한 특징을 탐지하게 됩니다. 이제 우리의 첫 번째 Conv2D layer를 살펴보겠습니다:
```Python
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same'...)
```
75라는 값은 우리가 학습하게 될 필터(Filter)의 갯수를 의미합니다. (3,3) 은 필터의 크기를 의미합니다. strides는 필터가 이미지를 얼마만큼의 간격으로 이동할 것인지를 나타냅니다. 마지막으로, padding은 입력 이미지와 결과 이미지의 크기를 맞추기 위해서 사용됩니다.

### [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) (배치정규화)

입력을 정규화하는 것과 마찬가지로, 배치정규화는 hidden layer들의 값을 scaling하여 학습을 개선합니다. [자세한 내용](https://blog.paperspace.com/busting-the-myths-about-batch-normalization/). 

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

<img src="images/maxpool2d.png" width=300 />
Max pooling은 이미지를 가져와서 더 낮은 해상도로 축소합니다. 이렇게 하면 모델이 약간의 변화에 더욱 견고하게 만들 수 있고, 모델의 학습 및 추론을 더욱 빠르게 할 수 있습니다.

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

<img src="images/dropout.png" width=360 />
드롭아웃은 과적합(Overfitting)을 방지하는 기술입니다. 드롭아웃은 무작위로 뉴런 중 일부를 선택하고 제외시켜,  뉴런들이 특정 경로에서 forward 혹은 backward progation에 참여하지 않도록 합니다. 이를 통해 네트워크가 한 영역에 의존하여 답을 도출하지 않도록 더욱 견고하고 중복적인 형태의 모델을 구성할 수 있습니다.

### [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) (평탄화)

Flatten은 다차원 입력을 1차원으로 변환시켜 줍니다. Flatten의 결과는 분류를 위한 마지막 layer로 연결되며, feature vector라고 불리게 됩니다.

### [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

이전 모델에서 이미 dense 레이어를 다룬 적이 있습니다. 현재 모델의 첫 dense 레이어 (512 유닛)는 feature vector를 입력으로 받으며, 어떠한 Feature가 분류에 기여하는지를 학습하게 됩니다. 두 번째 dense 레이어 (24 유닛)는 우리의 분류 예측 결과를 위한 마지막 layer로 활용됩니다.

## 모델 요약

다소 복잡하게 느껴질 수 있지만 걱정할 필요는 없습니다. CNN 모델을 효과적으로 훈련하기 위해 지금 당장 모든 것을 이해하는 것은 중요하지 않습니다. 가장 중요한 것은 이미지로부터 유용한 정보를 추출하는 데 도움이 되고 분류 작업에 사용될 수 있다는 것을 알고 계시면 충분합니다.

이제, 방금 만든 모델을 요약할 수 있습니다. 이전 노트북의 모델에 비해 학습가능(Trainable)한  매개 변수가 얼마나 적은지 알아 보십시오.

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 75)        750       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 75)        300       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        33800     
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 50)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 50)        200       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0

## Compiling the Model

이전의 예에서와 동일한 방식으로 모델을 컴파일할 수 있습니다.

In [9]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Training the Model

매우 다른 모델 구조에도 불구하고, 학습은 거의 비슷한 과정으로 진행됩니다. 아래 셀을 실행하여 20 에포크동안 학습하고, 정확도가 향상되는지 확인해 보겠습니다.

In [10]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/20
858/858 [==============================] - 5s 6ms/step - loss: 0.3075 - accuracy: 0.9058 - val_loss: 0.4242 - val_accuracy: 0.8747
Epoch 2/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0188 - accuracy: 0.9939 - val_loss: 0.1784 - val_accuracy: 0.9491
Epoch 3/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0090 - accuracy: 0.9970 - val_loss: 0.2660 - val_accuracy: 0.9229
Epoch 4/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0075 - accuracy: 0.9979 - val_loss: 0.1914 - val_accuracy: 0.9632
Epoch 5/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.3871 - val_accuracy: 0.9297
Epoch 6/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.4277 - val_accuracy: 0.9275
Epoch 7/20
858/858 [==============================] - 4s 5ms/step - loss: 0.0038 - accuracy: 0.9991 - val_loss: 0.2472 - val_accuracy: 0.9527
Epoch 


## 결과에 대한 논의

이 모델은 상당히 개선된 것 같습니다! 학습 정확도도 매우 높고, 검증 정확도도 향상되었습니다. 이것은 좋은 결과이며, 우리는 단지 모델을 새로운 모델로 바꾸기만 하면 됩니다.

추가적으로, 검증 정확도가 수렴하지 않는 것을 확인하셨을 수 있습니다. 이것은 여전히 우리 모델이 완벽히 일반화가 되지 않았다는 것을 의미합니다. 다행히도, 우리가 추가적으로 할 수 있는 것이 남아있습니다. 그것은 다음장에서 다루도록 하겠습니다.


## 요약

이 섹션에서는 CNN을 구현하기 위해 몇 가지 새로운 종류의 레이어를 활용하여, 이전 섹션에서 사용된 단순한 모델보다 더욱 높은 성능을 기록할 수 있었습니다. 준비된 데이터로 모델을 만들고 훈련하는 전반적인 과정에 더욱 익숙해지시기를 바랍니다.

## 메모리 지우기

넘어가기 전에 다음 셀을 실행하여 GPU 메모리를 지워주시기 바랍니다.

In [11]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## 다음

마지막 몇 가지 섹션에서는 모델의 생성 및 학습에 초점을 맞추었습니다. 이제 성능을 더욱 향상시키기 위해, 모델이 원래 학습하던 것보다 더 많고 좋은 데이터 제공을 위한 *데이터 증강(Augmentation)*이라 불리는 기법을 학습하도록 하겠습니다.